# Logistic Regression & Calibration — Student Lab

We focus on *probabilities*, not just accuracy.

In [32]:
import numpy as np

def check(name: str, cond: bool):
    if not cond:
        raise AssertionError(f'Failed: {name}')
    print(f'OK: {name}')

rng = np.random.default_rng(0)

## Section 0 — Synthetic imbalanced data
We simulate logits and labels with imbalance and miscalibration.

In [33]:
def make_probs(n=5000, base_rate=0.05, logit_scale=1.0, miscalibration=1.0):
    # Generate true probabilities via latent logits
    z = logit_scale * rng.standard_normal(n)
    # shift to get desired base rate approximately
    z = z + np.log(base_rate/(1-base_rate))
    p_true = 1/(1+np.exp(-z))
    y = (rng.random(n) < p_true).astype(int)
    # observed model probs are miscalibrated by scaling logits
    z_model = miscalibration * z
    p_model = 1/(1+np.exp(-z_model))
    return y, p_model, p_true

y, p_model, p_true = make_probs(miscalibration=2.0)
print('base_rate', y.mean())
check('in_0_1', np.all((p_model>=0) & (p_model<=1)))

base_rate 0.0712
OK: in_0_1


## Section 1 — Metrics

### Task 1.1: Confusion matrix metrics at a threshold
Implement precision, recall, F1 at threshold t.

# HINT:
- y_hat = (p>=t)
- TP/FP/FN

**Checkpoint:** Why is accuracy misleading under imbalance?

since the dataset is imbalanced, lets say 95 yes's to 5 no's. even if the model predicts all the outcomes as yes, it would still get 95% accuracy. so accuracy in an imbalanced dataset is misleading. we have to focus on no's as well.

in short, in an imbalced dataset, model is biased towards the majority class.

Hence we are using other metrices also to get better understanding.

In [34]:
def metrics_at_threshold(y, p, t):
    # TODO
    y = np.asarray(y, dtype=int)
    p = np.asarray(p, dtype=float)
    y_hat = (p > t).astype(int) # predicted labels based on threshold t
    
    tp = int(np.sum((y_hat == 1) & (y == 1))) # true positives
    tn = int(np.sum((y_hat == 0) & (y == 0))) # true negatives
    fp = int(np.sum((y_hat == 1) & (y == 0))) # false positives
    fn = int(np.sum((y_hat == 0) & (y == 1))) # false negatives
    
    # computing derived metrics using standard formulas
    accuracy = (tp + tn) / (len(y) + 1e-12) # to avoid division by zero
    precision = tp / (tp + fp + 1e-12) 
    recall = tp / (tp + fn + 1e-12)
    f1 = 2 * (precision * recall) / (precision + recall + 1e-12)
    return {
        'tp': tp,
        'tn': tn,
        'fp': fp,
        'fn': fn,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

m = metrics_at_threshold(y, p_model, t=0.5)
print(m)

{'tp': 2, 'tn': 4642, 'fp': 2, 'fn': 354, 'accuracy': 0.9287999999999998, 'precision': 0.499999999999875, 'recall': 0.005617977528089872, 'f1': 0.011111111111089075}


### Task 1.2: PR curve area (approx)
Compute a simple PR-AUC approximation by sorting thresholds.

# HINT:
- sort by p desc
- compute precision/recall at each cut

**Interview Angle:** when is PR-AUC preferable to ROC-AUC?

PR-AUC preferable to ROC-AUC when there is a huge data imbalance, true positives are our major focus area not true negetives. because PR-RUC does not use true negetives when computing. hence cases where we can ignore true negetives PR-AUC is preferable

In [35]:
def pr_curve(y, p):
    # TODO: return arrays (recall, precision)
    order = np.argsort(-p) # sorting indices by descending predicted probabilities
    y = np.asarray(y, dtype=int) # ensure y is an array
    y_sorted = y[order] # sorting y according to the sorted indices
    tp = np.cumsum(y_sorted==1) # cumulative true positives
    fp = np.cumsum(y_sorted==0) # cumulative false positives
    
    precision = tp / (tp + fp + 1e-12)
    recall = tp / (tp + tp[-1] + 1e-12)
    return recall, precision

def auc_trapz(x, y):
    # TODO
    x = np.asarray(x, dtype=float)
    y = np.asarray(y, dtype=float)
    return np.trapz(y, x) # integrating y with respect to x using trapezoidal rule
    
rec, prec = pr_curve(y, p_model)
pr_auc = auc_trapz(rec, prec)
print('pr_auc', pr_auc)
check('finite', np.isfinite(pr_auc))

pr_auc 0.12058854890412112
OK: finite


## Section 2 — Calibration

### Task 2.1: Reliability curve + ECE

Bin probabilities and compute:
- avg predicted prob per bin
- empirical accuracy per bin
- ECE = sum (bin_weight * |acc - conf|)

# HINT:
- np.digitize

**FAANG gotcha:** model can have good ranking but bad calibration.

In [36]:
def reliability_bins(y, p, n_bins=10):
    # TODO: return (bin_acc, bin_conf, bin_frac)
    y = np.asarray(y, dtype=int)
    p = np.asarray(p, dtype=float)
    bin_edges = np.linspace(0, 1, n_bins + 1)
    b = np.digitize(p, bin_edges[1:-1], right=False)
    
    bin_acc = np.zeros(n_bins)
    bin_conf = np.zeros(n_bins)
    bin_frac = np.zeros(n_bins)
    
    for i in range(n_bins):
        mask = (b == i)
        if mask.any():
            bin_acc[i] = y[mask].mean()
            bin_conf[i] = p[mask].mean()
            bin_frac[i] = mask.mean()
        else:
            bin_acc[i] = 0.0
            bin_conf[i] = 0.0
            bin_frac[i] = 0.0
    return bin_acc, bin_conf, bin_frac

def ece(bin_acc, bin_conf, bin_frac):
    # TODO
    return float(np.sum(np.abs(bin_acc - bin_conf) * bin_frac))

bin_acc, bin_conf, bin_frac = reliability_bins(y, p_model, n_bins=10)
ECE = ece(bin_acc, bin_conf, bin_frac)
print('ECE', ECE)
check('ece_finite', np.isfinite(ECE))

ECE 0.056267450617459955
OK: ece_finite


### Task 2.2: Temperature scaling

We assume p_model came from logits z_model. Approximate logits via logit(p).
Then find temperature T that minimizes NLL on validation split: sigmoid(z/T).

# HINT:
- logit(p)=log(p/(1-p))
- grid search T over [0.5..5]

**Checkpoint:** Why does scaling logits preserve ranking?

logits preserve ranking because in a montonic transformations all the elements are scaled by a same factor. so their relative order did not change. 

In [37]:
def logit(p, eps=1e-12):
    p = np.clip(p, eps, 1-eps)
    return np.log(p/(1-p))

def nll(y, p, eps=1e-12):
    p = np.clip(p, eps, 1-eps)
    return float(-np.mean(y*np.log(p) + (1-y)*np.log(1-p)))

# TODO: split into val and fit T
idx = rng.permutation(len(y))
val = idx[: len(y)//2]
test = idx[len(y)//2:]

z = logit(p_model)

Ts = np.linspace(0.5, 5.0, 50)
best_nll = float('inf')
best_T = None
for T in Ts:
    pT = 1/(1+np.exp(-(z[val]/T)))
    nll_T = nll(y[val], pT)
    if nll_T < best_nll:
        best_nll = nll_T
        best_T = T
        

print('best_T', best_T)
# apply temperature on test
p_cal = 1/(1+np.exp(-(z[test]/best_T)))
ECE_before = ece(*reliability_bins(y[test], p_model[test], 10))
ECE_after = ece(*reliability_bins(y[test], p_cal, 10))
print('ECE_before', ECE_before, 'ECE_after', ECE_after)

best_T 1.969387755102041
ECE_before 0.05789031077500049 ECE_after 0.006327853058047911


## Section 3 — Thresholding with costs

### Task 3.1: Pick threshold minimizing cost
Cost = c_fp*FP + c_fn*FN

# TODO: sweep thresholds and pick best.

**Interview Angle:** map model probabilities to business decisions.

Thresholding helps with cost but not accuracy. In cases where high caution is required and false positives are allowed to ceratain degree (like fraud detection) Thresholding is helpful.

In [38]:
def best_threshold_cost(y, p, c_fp=1.0, c_fn=10.0):
    # TODO
    y = np.asarray(y, dtype=int)
    p = np.asarray(p, dtype=float)
    ts = np.linspace(0, 1, 501)
    best_cost = float('inf')
    best_t = 0.5
    for t in ts:
        y_hat = (p > t).astype(int)
        fp = np.sum((y_hat == 1) & (y == 0))
        fn = np.sum((y_hat == 0) & (y == 1))
        cost = c_fp * fp + c_fn * fn
        if cost < best_cost:
            best_cost = cost
            best_t = float(t)
    return best_t, float(best_cost)        

t_star, cost_star = best_threshold_cost(y, p_model, c_fp=1.0, c_fn=10.0)
print('t*', t_star, 'cost', cost_star)

t* 0.006 cost 2626.0


---
## Submission Checklist
- All TODOs completed
- ECE computed
- Temperature scaling applied
- Threshold recommendation written
